# Subsetting an Unstructured Grid

In [ ]:
import uxarray as ux
import geoviews.feature as gf
import cartopy.crs as ccrs
import holoviews as hv

import geocat.datafiles as geodf


plot_opts = {"width" : 700, "height": 350}

hv.extension('bokeh')


In [ ]:
datafiles = (
    geodf.get(
        "netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/diag.2016-08-20_00.00.00_subset.nc"
    ),
    geodf.get("netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/x1.655362.grid_subset.nc"),
)

In [ ]:
uxds = ux.open_dataset(datafiles[1], datafiles[0])

In [ ]:
clim = (uxds['relhum_200hPa'][0].values.min(), uxds['relhum_200hPa'][0].values.max())

## Global Extent

In [ ]:
uxds['relhum_200hPa'][0].plot.rasterize(method='polygon', exclude_antimeridian=True, **plot_opts) * gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale='50m') * gf.states(projection=ccrs.PlateCarree(), line_width=1, scale='50m')

## Bounding Box

In [ ]:
lon_bounds = (-87.6298 - 2, -87.6298 + 2)
lat_bounds = (41.8781 - 2, 41.8781 + 2)

In [ ]:
bbox_subset = uxds['relhum_200hPa'][0].subset.bounding_box(lon_bounds, lat_bounds)

bbox_subset.plot.rasterize(method='polygon', exclude_antimeridian=True, clim=clim, **plot_opts) * gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale='50m') * gf.states(projection=ccrs.PlateCarree(), line_width=1, scale='50m')

## Bounding Circle

In [ ]:
center_coord = [-87.6298, 41.8781]

r = 2

In [ ]:
bcircl_subset = uxds['relhum_200hPa'][0].subset.bounding_circle(center_coord, r)

bcircl_subset.plot.rasterize(method='polygon', exclude_antimeridian=True, clim=clim, **plot_opts) * gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale='50m') * gf.states(projection=ccrs.PlateCarree(), line_width=1, scale='50m')

## Nearest Neighbor

In [ ]:
center_coord = [-87.6298, 41.8781]

k = 30

In [ ]:
nn_subset = uxds['relhum_200hPa'][0].subset.nearest_neighbor(center_coord, k)

nn_subset.plot.rasterize(method='polygon', exclude_antimeridian=True, clim=clim, **plot_opts) * gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale='50m') * gf.states(projection=ccrs.PlateCarree(), line_width=1, scale='50m')